In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd

from shapely.geometry import Point
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

ModuleNotFoundError: No module named 'seaborn'

In [2]:
hub_nash=pd.read_csv('../data/hub_requests.csv')

/Users/nicolemuldowney/opt/anaconda3/envs/geo/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
hub_nash.head()

,Request #,Status,Request Type,Subrequest Type,Additional Subrequest Type,Date / Time Opened,Date / Time Closed,Request Origin,System of Origin ID,Contact Type,...,State Issue,Parent Request,Closed When Created,Address,City,Council District,ZIP,Latitude,Longitude,Mapped Location
0,113016,Closed,"Trash, Recycling & Litter",Cart Service,Trash - Request an Additional Cart,10/30/2018 01:47:00 PM,03/05/2019 09:35:00 PM,Phone,NaN,NaN,...,False,NaN,False,2009 Upland Dr,NASHVILLE,7.0,37216,36.198638,-86.706659,"(36.1986378, -86.7066586)"
1,126185,Closed,"Streets, Roads & Sidewalks",Blocking the Right of Way,NaN,12/28/2018 01:04:00 PM,12/28/2018 01:04:00 PM,Phone,OE-00040610_12/27/2018 15:44,NaN,...,False,NaN,True,ELM HILL PKE / DONELSON PKE,NASHVILLE,15.0,37214,36.148809,-86.666282,"(36.14880891, -86.6662816)"
2,119344,Closed,"Trash, Recycling & Litter",Damage to Property,Trash Service,11/27/2018 02:20:00 PM,11/29/2018 04:04:00 AM,Phone,NaN,NaN,...,False,NaN,True,"597 Westchester Dr, Nashville, TN 37115, Unite...",MADISON,8.0,37115,36.263020,-86.738630,"(36.26302, -86.73863)"
3,127752,Closed,"Trash, Recycling & Litter",New Service/Activate Service,New Construction Residential Recycle,01/03/2019 06:57:00 PM,01/09/2019 10:09:00 AM,Phone,NaN,NaN,...,False,NaN,False,Ruffian Way,NaN,NaN,NaN,38.097609,-84.554097,"(38.0976091, -84.5540971)"
4,113477,Closed,"Trash, Recycling & Litter",Backdoor Trash or Recycle Waivers,Request Recycling Disability Waiver,10/31/2018 12:20:00 PM,10/31/2018 04:10:00 PM,Phone,NaN,NaN,...,False,NaN,True,"2326 Smith Springs Rd, Nashville, TN 37217, Un...",NASHVILLE,29.0,37217,36.096460,-86.637440,"(36.09646, -86.63744)"


Question 2 - Online vs phone by council district, could reveal differences in access?

In [2]:
hub_nash['Request Origin'].unique()

NameError: name 'hub_nash' is not defined

In [6]:
hub_nash['Request Origin'].value_counts()

Phone                     244656
hubNashville Community     88649
Mobile App                 13051
Email                       1721
Mayor's Office               372
Council Member               129
Council's Website             50
Walk-In                       18
Mayor's Website                8
In Person at Event             6
USPS Letter                    5
Social Media                   5
OEM Case                       1
Name: Request Origin, dtype: int64

Group request origin to online(hubNashville Community, Moblie App, Email, & Social Media), phone, and other

In [44]:
online_methods = ['hubNashville Community','Mobile App','Email','Social Media']
online_request_method = hub_nash[hub_nash['Request Origin'].isin(online_methods)]
online_request_method.shape

(103426, 21)

In [45]:
online_request_method['Council District'].value_counts().head(10)

19.0    9242
21.0    6138
6.0     5567
17.0    5126
24.0    4440
5.0     4350
20.0    4343
7.0     3425
25.0    3268
15.0    3187
Name: Council District, dtype: int64

In [48]:
online_request_method['City'].value_counts()  #should clean city data to aggregate/group and fix errors

NASHVILLE          76768
ANTIOCH             7759
MADISON             4301
HERMITAGE           2533
OLD HICKORY         1151
BRENTWOOD           1009
GOODLETTSVILLE       588
WHITES CREEK         323
JOELTON              256
Nashville            108
ASHLAND CITY          86
NOLENSVILLE           69
NASHVILLE, TN         43
PEGRAM                33
LA VERGNE             28
MOUNT JULIET          22
LAVERGNE              21
Antioch                9
FRANKLIN               7
ANTIOCH, TN            5
Hermitage              4
Nashville, TN          3
Nashvillle             2
BELLEVUE               2
NASHVILLE,TN           2
Madison                2
Antioch, TN            2
HERMITAGE, TN          2
Goodlettsville         1
Hermitage, TN          1
Mt. Juliet             1
Whites Creek           1
Nashvile, TN           1
Old Hickory            1
Nashville TN           1
nashville              1
UNKNOWN                1
Nashville,TN           1
OLD HICKORY, TN        1
Name: City, dtype: int64

In [34]:
#group phone request 
phone_request_method = hub_nash.loc[(hub_nash['Request Origin'] == "Phone")]
phone_request_method.shape

(244656, 21)

In [41]:
phone_request_method['Council District'].value_counts().head()

21.0    9765
19.0    9518
6.0     8961
5.0     8474
17.0    8319
Name: Council District, dtype: int64

In [33]:
phone_request_method['ZIP'].value_counts()

37013.0    13836
37211.0    12751
37209.0    11103
37207.0    10712
37206.0     9877
           ...  
37152.0        2
38011          1
37075.0        1
37232.0        1
0.0            1
Name: ZIP, Length: 78, dtype: int64

In [51]:
phone_request_method['City'].value_counts().head()

NASHVILLE    147044
ANTIOCH       17844
MADISON        9705
HERMITAGE      3039
BRENTWOOD      2436
Name: City, dtype: int64

In [38]:
#all other request orgins besides phone and web
other_methods = ["Mayor's Office",'Council Member',"Council's Website",'Walk-In', "Mayor's Website", 'In Person at Event','USPS Letter','Social Media', 'OEM Case']
other_request_method = hub_nash[hub_nash['Request Origin'].isin(other_methods)]
other_request_method.shape

(594, 21)

In [39]:
other_request_method['Council District'].value_counts()

21.0    28
9.0     24
5.0     22
8.0     22
19.0    21
7.0     20
24.0    20
1.0     19
16.0    18
33.0    18
2.0     13
31.0    13
29.0    13
6.0     13
25.0    11
34.0    11
3.0     10
23.0     9
20.0     9
17.0     8
27.0     8
4.0      8
28.0     8
15.0     8
30.0     5
14.0     5
18.0     5
13.0     4
12.0     4
32.0     4
22.0     4
11.0     2
35.0     2
26.0     2
10.0     1
Name: Council District, dtype: int64

In [42]:
other_request_method['ZIP'].value_counts().head(10) 
#need to clean some zip codes for no decimals and trailing 0 to aggregate these better

37115.0    34
37013.0    29
37211.0    27
37205.0    25
37208.0    24
37207.0    22
37209.0    18
37115      18
37013      18
37206.0    16
Name: ZIP, dtype: int64

In [50]:
other_request_method['City'].value_counts().head()

NASHVILLE    267
ANTIOCH       42
MADISON       39
Nashville     18
Madison       17
Name: City, dtype: int64